In [1]:
import os
import pandas as pd
from qdrant_client import QdrantClient, models
from fastembed import SparseTextEmbedding, LateInteractionTextEmbedding, MultiTaskTextEmbedding
# from src.embedding import JinaEmbeddings

DENSE_COLLECTION_NAME = "aaps_jina_vector"
HYBRID_COLLECTION_NAME = "aaps_hybrid_vector"

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("./data/dis-ademe-dispositifs-daide-r2da.csv")
cols = ["id", "titre", "description_courte", "description_longue"]

def clean_text(serie: pd.Series):
    serie = serie.str.replace(r"<[^>]+>", "", regex=True)
    serie = serie.str.replace("&nbsp;", " ", case=False)
    serie = serie.str.replace(r"’", "'", case=False)
    serie = serie.str.replace("l'ademe", "Elle", case=False)
    serie = serie.str.replace("ademe", "Elle", case=False)
    return serie


df.titre = clean_text(df.titre)
df.description_courte = clean_text(df.description_courte)
df.description_longue = clean_text(df.description_longue)
df[cols].head(1)

,id,titre,description_courte,description_longue
0,1ee7fafc-9b5a-6e4e-a950-855d180b6146,Mission de Commissionnement pour des rénovatio...,Pour vous aider à rénover vos bâtiments de man...,Pour sécuriser la qualité de votre rénovation ...


In [3]:
df['description'] = (df.titre + '\n' + df.description_courte + '\n' + df.description_longue)
documents = df[['id', 'description']].to_dict(orient="records")
df[:] = None
df = None
del df

/var/folders/pl/980lv6h92qg2ffgp6b0rw65h0000gn/T/ipykernel_12779/4240891094.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[:] = None
/var/folders/pl/980lv6h92qg2ffgp6b0rw65h0000gn/T/ipykernel_12779/4240891094.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[:] = None


## Dense only

In [ ]:
jina = JinaEmbeddings()
task = "retrieval.passage"

In [ ]:
qdrant_client = QdrantClient(
    host=os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

In [ ]:
# qdrant_client.create_collection(
#     collection_name=COLLECTION_NAME,
#     vectors_config=models.VectorParams(
#         size=jina.vector_size,
#         distance=models.Distance.COSINE,
#     ),
# )

### Index

In [ ]:
qdrant_client.upload_points(
    collection_name=DENSE_COLLECTION_NAME,
    points=[
        models.PointStruct(
            id=doc["id"],
            vector=jina.encode(doc["description"], task)[0].tolist(),
            payload=doc,
        )
        for doc in documents
    ],
)

### Search

In [ ]:
query_text = "Remplacement de mon groupe froid"
hits = qdrant_client.query_points(
    collection_name=DENSE_COLLECTION_NAME,
    query=jina.encode(query_text,task="retrieval.query").tolist()[0],
    limit=10,
).points

for hit in hits:
    print(hit.payload, "\nscore:", hit.score)

## Hybrid

In [4]:
qdrant_client = QdrantClient(
    host=os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

In [ ]:
# jina= JinaEmbeddings()
jina = MultiTaskTextEmbedding(model_name="jinaai/jina-embeddings-v3")
task = "retrieval.passage"

bm42 = SparseTextEmbedding(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions")

def embed_with_bm42(passage: str) -> models.SparseVector:
    embedding = list(bm42.passage_embed(passage))[0]
    return models.SparseVector(
                indices=embedding.indices.tolist(),
                values=embedding.values.tolist()
            )
colbert_model_name = "colbert-ir/colbertv2.0"
colbert = LateInteractionTextEmbedding(colbert_model_name)

### Index

In [ ]:
colbert_vector_size = LateInteractionTextEmbedding._get_model_description(colbert_model_name)["dim"]

qdrant_client.recreate_collection(
    collection_name=HYBRID_COLLECTION_NAME,
    vectors_config={
        "jina_dense": models.VectorParams(
            size=jina.vector_size,
            distance=models.Distance.COSINE,
        ),
        "jina_colbert": models.VectorParams(
            size=colbert_vector_size,
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM
            ),
        ),
    },
    sparse_vectors_config={
        "bm42": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    },
)

In [ ]:
qdrant_client.upload_points(
    collection_name=HYBRID_COLLECTION_NAME,
    points=[
        models.PointStruct(
            id=doc["id"],
            vector={
                "jina_dense": jina.encode(doc["description"], task)[0].tolist(),
                "jina_colbert": list(colbert.passage_embed(doc["description"]))[0],
                "bm42": embed_with_bm42(documents[0]["description"]),
            },
            payload=doc,
        )
        for doc in documents
    ],
    batch_size=16,
)

### Search

In [5]:
jina = MultiTaskTextEmbedding(model_name="jinaai/jina-embeddings-v3")
bm42 = SparseTextEmbedding(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions")

def embed_with_bm42(passage: str) -> models.SparseVector:
    embedding = list(bm42.passage_embed(passage))[0]
    return models.SparseVector(
                indices=embedding.indices.tolist(),
                values=embedding.values.tolist()
            )
colbert_model_name = "colbert-ir/colbertv2.0"
colbert = LateInteractionTextEmbedding(colbert_model_name)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
query_text = "Je rénove un entrepot logistique et veux remplacer mon groupe froid de 500 kW."

sparse_embedding = list(bm42.query_embed(query_text))[0]
dense_embedding = list(jina.task_embed(documents[0]["description"], task_type = "retrieval.query"))[0].embedding
colbert_embedding = list(colbert.query_embed(query_text))[0]

hits = qdrant_client.query_points(
  collection_name=HYBRID_COLLECTION_NAME,
  prefetch=[
      models.Prefetch(query=sparse_embedding.as_object(), using="bm42", limit=15),
      models.Prefetch(query=dense_embedding.tolist(), using="jina_dense", limit=15),
  ],
  query=colbert_embedding.tolist(),
  using="jina_colbert",
  limit=10,
).points

for hit in hits:
    print(hit.payload, "\nscore:", hit.score)

{'id': '1edbbf5b-5ef2-6a26-89fc-8180f3010746', 'description': "Études de réseaux de chaleur renouvelables, 2ᵉ édition : jusqu'à 80 % de vos études financées\nCet Appel à projets propose une aide exceptionnelle pour vous aider à identifier et mettre en œuvre les projets de réseaux de chaleur renouvelable et de récupération ou de boucles d'eau tempérées géothermique les plus pertinents sur votre territoire.\nVous souhaitez réunir les conditions de réussite par l'intermédiaire d'une étude ? Vous souhaitez faire appel à un bureau d'étude externe ou un cabinet conseil indépendant ? Votre collectivité couvre une population inférieure à 50 000 habitants ?Elle pourrait financer votre étude jusqu'à 80 % (via le principe de minimis).Dans un deuxième temps, elle pourra offrir des aides Fonds Chaleur pour vos travaux de réalisation.Elle met à disposition des modèles de cahier des charges et des guides pour faciliter l'appel à un bureau d'étude ou à un cabinet conseil.Vous vous interrogez sur la pe